In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [7]:
import pandas as pd

train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

print(train.shape, test.shape)
train.head()


(891, 12) (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [5]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [8]:
print(train.groupby('Sex')['Survived'].mean())
print(train.groupby('Pclass')['Survived'].mean())
print(train.groupby(['Sex','Pclass'])['Survived'].mean())


Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64
Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64
Sex     Pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: Survived, dtype: float64


In [9]:
def logic_rule(row):
    if row['Sex'] == 'female':
        if row['Pclass'] == 3 and row['Age'] > 30:
            return 0   # older 3rd-class females less likely
        else:
            return 1
    else:
        if row['Pclass'] == 1 and row['Age'] < 10:
            return 1   # young 1st-class boys had a chance
        else:
            return 0

test['Survived'] = test.apply(logic_rule, axis=1)


In [10]:
submission = test[['PassengerId', 'Survived']]
submission.to_csv("submission.csv", index=False)
submission.head()


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [12]:
# --- Titanic Baseline Notebook ---
# Author: Abhinay Shankar Belde
# File: baseline.ipynb
# Goal: Rule-based baseline (no ML model)

import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report

# ---------------- Load data ----------------
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test  = pd.read_csv("/kaggle/input/titanic/test.csv")

# ---------------- Basic pattern exploration ----------------
print("Survival rate by Sex:")
print(train.groupby('Sex')['Survived'].mean(), "\n")

print("Survival rate by Pclass:")
print(train.groupby('Pclass')['Survived'].mean(), "\n")

print("Survival rate by Sex and Pclass:")
print(train.groupby(['Sex','Pclass'])['Survived'].mean(), "\n")

# ---------------- Fill missing ages ----------------
train['Age'].fillna(train['Age'].median(), inplace=True)
test['Age'].fillna(test['Age'].median(), inplace=True)

# ---------------- Logic-based rule (Option B) ----------------
def logic_rule(row):
    if row['Sex'] == 'female':
        if row['Pclass'] == 3 and row['Age'] > 30:
            return 0   # older 3rd-class females less likely
        else:
            return 1
    else:
        if row['Pclass'] == 1 and row['Age'] < 10:
            return 1   # young 1st-class boys had a chance
        else:
            return 0

# ---------------- Evaluate on training data ----------------
train['Predicted'] = train.apply(logic_rule, axis=1)

acc = accuracy_score(train['Survived'], train['Predicted'])
f1  = f1_score(train['Survived'], train['Predicted'])
cm  = confusion_matrix(train['Survived'], train['Predicted'])

print("=== Baseline (Logic Rule) Performance on Training Set ===")
print(f"Accuracy: {acc:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(train['Survived'], train['Predicted'], digits=4))

# ---------------- Generate Kaggle submission ----------------
test['Survived'] = test.apply(logic_rule, axis=1)
submission = test[['PassengerId','Survived']]
submission.to_csv("submission.csv", index=False)

print("\nSubmission file created: submission.csv")
submission.head()


Survival rate by Sex:
Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64 

Survival rate by Pclass:
Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64 

Survival rate by Sex and Pclass:
Sex     Pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: Survived, dtype: float64 

=== Baseline (Logic Rule) Performance on Training Set ===
Accuracy: 0.7991
F1 Score: 0.7181
Confusion Matrix:
 [[484  65]
 [114 228]]

Classification Report:
               precision    recall  f1-score   support

           0     0.8094    0.8816    0.8439       549
           1     0.7782    0.6667    0.7181       342

    accuracy                         0.7991       891
   macro avg     0.7938    0.7741    0.7810       891
weighted avg     0.7974    0.7991    0.7956       891


Submission file created: submission.csv


/tmp/ipykernel_37/1336008916.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Age'].fillna(train['Age'].median(), inplace=True)
/tmp/ipykernel_37/1336008916.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [13]:
import os
os.listdir("/kaggle/working")


['submission.csv', '.virtual_documents']

In [14]:
from IPython.display import FileLink
FileLink('submission.csv')


/kaggle/working/submission.csv